In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
data = open("muse_all_songs.txt").read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print("word count:", total_words)


{'you': 1, 'the': 2, 'and': 3, 'i': 4, 'to': 5, 'me': 6, 'a': 7, 'your': 8, 'it': 9, 'in': 10, 'of': 11, 'will': 12, 'my': 13, 'be': 14, 'we': 15, "i'm": 16, 'is': 17, 'can': 18, 'this': 19, "don't": 20, "it's": 21, 'for': 22, 'all': 23, "can't": 24, 'are': 25, 'on': 26, 'that': 27, "you're": 28, 'no': 29, 'us': 30, 'but': 31, 'just': 32, 'so': 33, 'never': 34, 'our': 35, 'now': 36, 'out': 37, 'pro': 38, "i'll": 39, 'feel': 40, 'from': 41, 'one': 42, 'know': 43, 'up': 44, 'with': 45, 'want': 46, 'when': 47, 'have': 48, 'make': 49, 'could': 50, 'away': 51, 'love': 52, 'oh': 53, 'let': 54, 'down': 55, 'gonna': 56, 'need': 57, 'too': 58, 'yeah': 59, "won't": 60, 'they': 61, 'what': 62, "you've": 63, 'who': 64, 'world': 65, 'see': 66, 'break': 67, 'do': 68, 'not': 69, 'time': 70, 'been': 71, 'ah': 72, 'like': 73, 'life': 74, 'come': 75, 'how': 76, 'free': 77, 'wrong': 78, 'give': 79, 'get': 80, "i've": 81, 'people': 82, 'am': 83, 'take': 84, 'fight': 85, 'run': 86, 'was': 87, 'control': 88

In [18]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(total_words, 128, input_length=max_sequence_len - 1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

adam = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


earlystop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

history = model.fit(xs, ys, epochs=150, verbose=1, callbacks=[earlystop])


In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
seed_text = "your sentence"
next_words = 100

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)